<a href="https://colab.research.google.com/github/jslonpl/Test_Git/blob/main/image_scaling_opencv_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv
import os
import numpy as np

In [ ]:
import os
dir_path = 'drive/MyDrive/Colab Notebooks/clear_line/img'
img_obj = []
for img_No, img_name in enumerate (os.listdir(dir_path)):
  img_obj.append(os.path.join(dir_path, img_name))

drive/MyDrive/Colab Notebooks/clear_line/img/0.jpg


Creating list, that contains 4 ideal images ( visible 5 ideal rows ) from directory.

In [ ]:
dir_path = 'drive/MyDrive/Colab Notebooks/clear_line/img/ideal_5_rows'

def load_image_from_directory(dir_path):
  images = []
  for filename in os.listdir(dir_path):
    if filename.endswith('.jpg'):
      img_path = os.path.join(dir_path, filename)
      #img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
      img = cv.imread(img_path)
      if img is not None:
        images.append(img)
  return images

In [ ]:
from PIL import Image
from IPython.display import display

def colab_imshow(img, text):
  image_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
  pil_img = Image.fromarray(image_rgb)
  display(str(text),pil_img)

In order to show img with using google colab use function : colab_imshow(), where img is image.jpg

In [ ]:
images = load_image_from_directory(dir_path)
colab_imshow(images[2], "test")

Characteristic points SIFT detector initialization

In [ ]:
import cv2 as cv
import numpy as np

#dir_path = 'drive/MyDrive/Colab Notebooks/clear_line/img'
dir_path = 'drive/MyDrive/Colab Notebooks/clear_line/img/ideal_5_rows'

# Wczytanie 12 zdjęć (zakładając, że mają nazwy image1.png, image2.png, ..., image12.png)
images = load_image_from_directory(dir_path)

# Inicjalizacja detektora punktów charakterystycznych SIFT
sift = cv.SIFT_create()

# Wykrycie punktów charakterystycznych na wszystkich obrazach
keypoints_list = []
descriptors_list = []
for image in images:
    keypoints, descriptors = sift.detectAndCompute(image, None)
    keypoints_list.append(keypoints)
    descriptors_list.append(descriptors)

# Dopasowanie punktów charakterystycznych między parą obrazów
matcher = cv.BFMatcher()
matches_list = []
for i in range(len(images) - 1):
    matches = matcher.knnMatch(descriptors_list[i], descriptors_list[i + 1], k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    matches_list.append(good_matches)

# Przygotowanie punktów charakterystycznych do obliczenia macierzy transformacji
src_pts_list = []
dst_pts_list = []
for i, matches in enumerate(matches_list):
    src_pts = np.float32([keypoints_list[i][m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints_list[i + 1][m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    src_pts_list.append(src_pts)
    dst_pts_list.append(dst_pts)

# Obliczenie macierzy transformacji afinicznej (macierzy homografii) dla każdej pary obrazów
M_list = []
for src_pts, dst_pts in zip(src_pts_list, dst_pts_list):
  M, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
  M_list.append(M)

# Wybranie obrazu do przeprowadzenia transformacji perspektywicznej (np. image1.png)
chosen_image_idx = 0
chosen_image = images[chosen_image_idx]

# Przeprowadzenie transformacji perspektywicznej na wybranym obrazie
output_image = chosen_image
for i, M in enumerate(M_list):
    if i < chosen_image_idx:
        output_image = cv.warpPerspective(output_image, M, (output_image.shape[1], output_image.shape[0]))
    elif i > chosen_image_idx:
        M_inv = np.linalg.inv(M)
        output_image = cv.warpPerspective(output_image, M_inv, (output_image.shape[1], output_image.shape[0]))

# Wyświetlanie wyniku
colab_imshow(chosen_image, "WYbrany obraz przed transformacją")
colab_imshow(output_image, 'Obraz po transformacji')

Test

In [ ]:
# Wybranie obrazu do przeprowadzenia transformacji perspektywicznej (np. image1.png)
dir_path = 'drive/MyDrive/Colab Notebooks/clear_line/img'
images = load_image_from_directory(dir_path)
chosen_image_idx = 9
chosen_image = images[chosen_image_idx]

# Przeprowadzenie transformacji perspektywicznej na wybranym obrazie
output_image = chosen_image
for i, M in enumerate(M_list):
    if i < chosen_image_idx:
        output_image = cv.warpPerspective(output_image, M, (output_image.shape[1], output_image.shape[0]))
    elif i > chosen_image_idx:
        M_inv = np.linalg.inv(M)
        output_image = cv.warpPerspective(output_image, M_inv, (output_image.shape[1], output_image.shape[0]))

# Wyświetlanie wyniku
colab_imshow(chosen_image, "WYbrany obraz przed transformacją")
colab_imshow(output_image, 'Obraz po transformacji')

Ostateczny rezultat transformacji nie jest satysfakcjonujący.

Należy przeprowadzić dodatkowy research w celu znalezienia lepszej metody bądź dostosowania aktualnej.